# Описание задания
### Задание
Улучшите рекомендательную систему с помощью информации о поле клиентов
### Датасет
- `purchases_train.csv` - история покупок в розничном магазине с 21 октября 2003 года по 12 марта 2004 года.
- `purchases_test.csv` - покупки за следующую неделю, с 13 по 19 марта 2004 года. В этой выборке для каждого пользователя исключены товары, которые он уже покупал за период обучающей выборки. Так сделано, чтобы измерить, насколько модель хорошо предсказывает будущие покупки.
- `customers.csv` - пол клиентов (`Male` / `Female`), есть пропуски
### Бейзлайн решение
- на истории покупок обучается модель матричного разложения `implicit.AlternateLeastSquares`
- рекомендации сравниваются с покупками за следующую неделю по метрике `map@10`

# Выгрузка данных, бейзлайн решение, метрика качества

In [1]:
# you need implicit & ml_metrics packages to run this notebook
# pip3 install implicit ml_metrics


from pathlib import Path

import numpy as np
from IPython.core.display import display
from ml_metrics import mapk
import scipy
import pandas as pd
import implicit

from src.load import DataProvider
from src.baseline_model import get_baseline_model
from src.utils import transform_to_item_user_csr_matrix, get_recommendations, get_purchases_by_customer
import inspect

In [2]:
data_provider = DataProvider(data_directory=Path('./data'))
purchases_train = data_provider.get_purchases_train()
purchases_test = data_provider.get_purchases_test()
display(
    purchases_train.head(),
    purchases_test.head(),
)

,customer_id,product_id,datetime
0,8698595,12530,2004-03-10 22:18:43.497459200
1,13271885,7541,2004-03-06 02:24:43.209763200
2,16852746,13134,2004-03-10 01:03:09.598614400
3,16852746,6572,2004-03-04 16:45:16.522566400
4,14619070,4659,2004-03-12 13:29:35.011481600


,customer_id,product_id,datetime
0,1021292,6197,2004-03-18 13:35:19.145152000
1,11379978,4659,2004-03-19 18:51:31.887936000
2,13271885,5659,2004-03-14 05:47:21.544166400
3,13271885,1015,2004-03-15 14:41:19.702089601
4,12315337,12072,2004-03-19 10:39:17.148105600


Бейзлайн решение основано на библитеке матричного разложения `implicit`.

In [3]:
print(inspect.getsource(get_baseline_model))

def get_baseline_model() -> implicit.als.AlternatingLeastSquares:
    # disable internal multithreading to speed up implicit.als.AlternatingLeastSquares.fit()
    environ["MKL_NUM_THREADS"] = "1"
    environ["OPENBLAS_NUM_THREADS"] = "1"

    # we iterated through hyper parameters and measured map@10 score on test set
    # the parameters below provide 80-th percentile of score
    # we intentionally do not use parameters with best test score
    model = implicit.als.AlternatingLeastSquares(
        factors=20, iterations=7, regularization=100.0
    )
    return model



In [4]:
def train_model_on_purchases_and_measure_map10_score(unfitted_model: implicit.als.AlternatingLeastSquares) -> float:
    data_provider = DataProvider(data_directory=Path('./data'))
    item_users = transform_to_item_user_csr_matrix(data_provider.get_purchases_train())
    
    np.random.seed(42)
    unfitted_model.fit(item_users=item_users)
    
    test_user_ids, test_purchases = get_purchases_by_customer(data_provider.get_purchases_test())
    recommendations = get_recommendations(unfitted_model, test_user_ids, item_users)
    score = mapk(test_purchases, recommendations, k=10)
    return score

baseline_score = train_model_on_purchases_and_measure_map10_score(unfitted_model=get_baseline_model())
print(baseline_score)


0.18293975547686622


# Улучшите модель с помощью данных о поле потребителя

In [3]:
customers = data_provider.get_customers()
display(customers.head())

,customer_id,sex
0,14386819,Female
1,1481405,NaN
2,16745074,NaN
3,10325906,NaN
4,11167384,NaN


In [4]:
# подсчет количества нулевых значений
customers.isnull().sum()

customer_id        0
sex            89089
dtype: int64

In [5]:
# заполнение нулевых значений
customers.fillna('unknown', inplace=True)

In [6]:
from sklearn import preprocessing

In [7]:
# перекодировка значений в числовые и добавление информации о поле в тренировочный и тестовый датафреймы
customers['sex'] = preprocessing.LabelEncoder().fit_transform(customers['sex'])
purchases_train = purchases_train.join(customers.set_index('customer_id'), on='customer_id')
purchases_test = purchases_test.join(customers.set_index('customer_id'), on='customer_id')

In [8]:
purchases_train.head()

,customer_id,product_id,datetime,sex
0,8698595,12530,2004-03-10 22:18:43.497459200,2
1,13271885,7541,2004-03-06 02:24:43.209763200,0
2,16852746,13134,2004-03-10 01:03:09.598614400,0
3,16852746,6572,2004-03-04 16:45:16.522566400,0
4,14619070,4659,2004-03-12 13:29:35.011481600,0


Вариант решения, который я вижу - обучить отдельные модели для женщин и мужчин и сравнить их результаты с результатами базового алгоритма на этих же покупателях

In [34]:
def compare_train_model_on_purchases_and_measure_map10_score_by_sex(model, unfitted_model, sex) -> float:
    item_users = transform_to_item_user_csr_matrix(purchases_train[purchases_train['sex'] == sex])
    item_users_base = transform_to_item_user_csr_matrix(data_provider.get_purchases_train())
    
    np.random.seed(42)
    model.fit(item_users=item_users)
    unfitted_model.fit(item_users=item_users_base)
    
    test_user_ids, test_purchases = get_purchases_by_customer(purchases_test[purchases_test['sex'] == sex])
    recommendations = get_recommendations(model, test_user_ids, item_users)
    recommendations_base = get_recommendations(unfitted_model, test_user_ids, item_users)
    score = mapk(test_purchases, recommendations, k=10)
    base_score = mapk(test_purchases, recommendations_base, k=10)
    return score, base_score

In [35]:
female_scores = compare_train_model_on_purchases_and_measure_map10_score_by_sex(
    implicit.als.AlternatingLeastSquares(), 
    implicit.als.AlternatingLeastSquares(), 0)
print(female_scores)


(0.08952842937473722, 0.07391380127270607)


In [36]:
male_scores = compare_train_model_on_purchases_and_measure_map10_score_by_sex(
    implicit.als.AlternatingLeastSquares(), 
    implicit.als.AlternatingLeastSquares(), 1)
print(male_scores)


(0.07152967130349548, 0.05983730164551614)


In [37]:
unknown_scores = compare_train_model_on_purchases_and_measure_map10_score_by_sex(
    implicit.als.AlternatingLeastSquares(), 
    implicit.als.AlternatingLeastSquares(), 2)
print(unknown_scores)


(0.022697501438435172, 0.10391554414368606)


Результаты ожидаемые: алгоритмы, обучающиеся на покупателях одного пола, лучше справляются с задачей, чем базовый алгоритм. А вот для покупателей, чей пол неизвестен, лучше работает базовый, потому что оба алгоритма не имеют никакой информации о поле, однако базовый алгоритм обучался на большей выборке. Поэтому для покупателей, пол которых известен, я бы использовал модели по полу, а для остальных - общую модель по всей выборке

In [80]:
import itertools

In [84]:
parametrs = {'factors' : [5, 10],
            'regularization': [0.01, 0.001]}

In [114]:
# Функция для подбора параметров
def grid_search(model, parametrs, train, test):
    item_users = transform_to_item_user_csr_matrix(purchases_train)
    df = pd.DataFrame(columns=['score', 'parameters'])
    n=0
    np.random.seed(42)
    
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        
        model=implicit.als.AlternatingLeastSquares(**params)
        model.fit(item_users=item_users)        
        item_users = transform_to_item_user_csr_matrix(train)
   
        test_user_ids, test_purchases = get_purchases_by_customer(test)
        recommendations = get_recommendations(model, test_user_ids, item_users)
        score = mapk(test_purchases, recommendations, k=10)
        
        df.loc[n] = [score, params]
        n += 1
        
    return df.sort_values(by=['score'],ascending=False)
    

In [115]:
grid_search(implicit.als.AlternatingLeastSquares, parametrs, purchases_train, purchases_test)

,score,parameters
3,0.143883,"{'factors': 10, 'regularization': 0.001}"
2,0.143865,"{'factors': 10, 'regularization': 0.01}"
0,0.139347,"{'factors': 5, 'regularization': 0.01}"
1,0.139288,"{'factors': 5, 'regularization': 0.001}"


Реализовал функцию для подбора параметров. В дальнейшем ее также можно легко адаптировать под алгоритмы для разных полов. Для демонстрации взял низкие значения параметра 'factors' для быстрой проверки работы функции
